In [35]:
import numpy as np
import pandas as pd
import itertools
from sklearn import preprocessing
import matplotlib.pyplot as plt
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.learn.python.learn import metric_spec
from tensorflow.contrib.learn.python.learn.estimators import _sklearn
from tensorflow.contrib.learn.python.learn.estimators import estimator
from tensorflow.contrib.learn.python.learn.estimators import model_fn
from tensorflow.python.framework import ops
from tensorflow.python.saved_model import loader
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.util import compat
tf.logging.set_verbosity(tf.logging.FATAL) 

In [36]:
data = pd.read_csv('/home/alam/Downloads/New DAta/DataNew.csv')


## Creating the Target Label

From a prior notebook, I examined the 'loan_status' column. The cell below creates a column with binary value 0 for loans not in default, and binary value 1 for loans in default.


In [37]:
data['Default_Binary'] = int(0)
for index, value in data.Credit_History.iteritems():
    if value == 'Default':
        data.set_value(index,'Default_Binary',int(1))
    if value == 'Charged Off':
        data.set_value(index, 'Default_Binary',int(1))
    if value == 'Late (31-120 days)':
        data.set_value(index, 'Default_Binary',int(1))    
    if value == 'Late (16-30 days)':
        data.set_value(index, 'Default_Binary',int(1))
    if value == 'Does not meet the credit policy. Status:Charged Off':
        data.set_value(index, 'Default_Binary',int(1))  


## Creating a category feature for "Loan Purpose"

Below I create a new column for loan purpose, and assign each type of loan purpose an integer value.


In [38]:
data['Purpose_Cat'] = int(0) 
for index, value in data.Purpose_Credit_Taken.iteritems():
    if value == 'Vehicle':
        data.set_value(index,'Purpose_Cat',int(1))
    if value == 'Furniture':
        data.set_value(index, 'Purpose_Cat',int(2))
    if value == 'Electronics Appliances':
        data.set_value(index, 'Purpose_Cat',int(3))    
    if value == 'Vacation':
        data.set_value(index, 'Purpose_Cat',int(4))    
    if value == 'Repairs':
        data.set_value(index,'Purpose_Cat',int(5))
    if value == 'Remodeling house':
        data.set_value(index, 'Purpose_Cat',int(6))
    if value == 'Education':
        data.set_value(index, 'Purpose_Cat',int(7))    
    if value == 'Training':
        data.set_value(index, 'Purpose_Cat',int(8))   
    if value == 'Business':
        data.set_value(index, 'Purpose_Cat',int(9))    
    if value == 'Others':
        data.set_value(index,'Purpose_Cat',int(10))


## Scaling Loan Amount

Below I scale the amount funded for each loan to a value between 0 and 1


In [39]:
x = np.array(data.Credit_Amount.values).reshape(-1,1) 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data['Credit_Amount_scaled'] = pd.DataFrame(x_scaled)
print (data.Credit_Amount_scaled[0:5])

0    0.202982
1    0.085012
2    0.324254
3    0.133432
4    0.667932
Name: Credit_Amount_scaled, dtype: float64



## Scaling Number of Credit

Below I scale the interest rate for each loan to a value between 0 and 1


In [40]:
x = np.array(data.Num_Credits.values).reshape(-1,1) 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data['Num_Credits_scaled'] = pd.DataFrame(x_scaled)
print (data.Num_Credits_scaled[0:5])


0    0.333333
1    0.333333
2    0.333333
3    0.000000
4    0.000000
Name: Num_Credits_scaled, dtype: float64



## Setting up the Neural Network

Below I split the data into a training, testing, and prediction set

After that, I assign the feature and target columns, and create the function that will be used to pass the data into the model


In [42]:
training_set = data[0:3000] 
testing_set = data[3001:4000] 
prediction_set = data[4001:] 

In [43]:
COLUMNS = ['Purpose_Cat','Credit_Amount_scaled','Num_Credits_scaled','Default_Binary']          
FEATURES = ['Purpose_Cat','Credit_Amount_scaled','Num_Credits_scaled']
LABEL = 'Default_Binary'


In [44]:
def input_fn(data_set):
    feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES} 
    labels = tf.constant(data_set[LABEL].values)
    return feature_cols, labels

## Fitting The Model 

In [46]:
feature_cols = [tf.contrib.layers.real_valued_column(k)
              for k in FEATURES]
#config = tf.contrib.learn.RunConfig(keep_checkpoint_max=1) ######## DO NOT DELETE
regressor = tf.contrib.learn.DNNRegressor(
  feature_columns=feature_cols, hidden_units=[10, 20, 10], ) 
regressor.fit(input_fn=lambda: input_fn(training_set), steps=251)


DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x7f748ec12450>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='Purpose_Cat', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='Credit_Amount_scaled', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='Num_Credits_scaled', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)), 'embedding_lr_multipliers': None, 'optimizer': None, 'dropout': None, 'gradient_clip_norm': None, 'activation_fn': <function relu at 0x7f74ac929d70>, 'input_layer_min_slice_size': None})

## Evaluating the Model 

In [47]:
# Score accuracy
ev = regressor.evaluate(input_fn=lambda: input_fn(testing_set), steps=10)
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))


Loss: 0.000024


## Predicting on new data

In [77]:
y = regressor.predict(input_fn=lambda: input_fn(prediction_set))
predictions = list(itertools.islice(y,1000))
